### Imports

In [1]:
import json
from collections import Counter
import pandas as pd
import nltk
import emoji
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Carregando Base Atual

In [2]:
# Ler Base de dados
database = pd.read_json("../base_completa/boatos_br_corpus.json")
# database = pd.read_json("./base_atualizada/boatos_br_corpus_atualizada.json")
database.shape

corpus_text = database['texto-normalizado'].tolist()

In [3]:
# TOKENIZATION
corpus_tokenized = [sentence.split() for sentence in corpus_text]

In [4]:
# Data Extractor

def extractEmojis(text:str):
    emoji_pattern = re.compile(
        r'['
        u'\U0001F600-\U0001F64F'  # Emoticons
        u'\U0001F300-\U0001F5FF'  # Símbolos e pictogramas
        u'\U0001F680-\U0001F6FF'  # Transportes e símbolos de mapa
        u'\U0001F1E0-\U0001F1FF'  # Bandeiras (iOS)
        u'\U00002500-\U00002BEF'  # Formas geométricas
        u'\U00002702-\U000027B0'  # Símbolos adicionais
        u'\U000024C2-\U0001F251'  # Símbolos miscelâneos
        u'\U0001f926-\U0001f937'  # Gestos
        u'\U00010000-\U0010FFFF'  # Caracteres suplementares
        u'\u200d'  # Zero width joiner
        u'\u2640-\u2642'  # Símbolos de gênero
        u'\u2600-\u2B55'  # Diversos símbolos e pictogramas
        u'\u23cf'  # Símbolo de ejeção
        u'\u23e9'  # Símbolos de controle de mídia
        u'\u231a'  # Relógios
        u'\ufe0f'  # Variações de caracteres
        u'\u3030'  # Símbolo de alternância
        ']+', 
        flags=re.UNICODE
    )
    emojis = re.findall(emoji_pattern, text)
    return [list(_emoji) for _emoji in emojis]
    # return [item for sublist in  [list(_emoji) for _emoji in emojis] for item in sublist]

# Palavras em Maiusculo
def getNumberWordsUppercase(text:list[str]):
    words_in_uppercase = [word for word in text if word.isupper()]
    return len(words_in_uppercase)

# Quantidade de emojis
def getNumberEmojis(text:list[str]):
    return len([char for char in text if char in emoji.EMOJI_DATA])

def getNumberVerbs(text:str):
    palavras = nltk.tokenize.word_tokenize(text)
    pos_tags = nltk.pos_tag(palavras)
    return len([word for word, pos in pos_tags if pos.startswith('VB')])

In [5]:
database_dict = database.to_dict('records')

# extractEmojis(database_dict[456]['texto'])
extractEmojis(database_dict[470]['texto'])

[['\U0001faf5', '🏼'], ['🇧', '🇷', '🚨'], ['💣'], ['💩'], ['🪖']]

In [6]:
for data in database_dict:
    text = data['texto'].split()
    data['num-emojis'] = len(extractEmojis(data['texto']))
    data['num-verbos'] = getNumberVerbs(data['texto'])
    data['num-palavras'] = len(text)
    data['num-palavras-maiusculas'] = getNumberWordsUppercase(text)    

In [7]:

with open(f"../base_normalizada/boatos_br_corpus_normalizado.json",'w+',encoding="utf8") as f:
    json.dump(database_dict,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()